### Installing Required Libraries

In [1]:
#!conda install -c conda-forge beautifulsoup4 --yes
#!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge nltk --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The fo

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from math import sqrt

import requests # library to handle requests

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

###!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
###!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



# to access wikipedia pages and perform required clenaup
##from bs4 import BeautifulSoup
##import lxml

print('Libraries imported.')

Libraries imported.


### Foursquare credentials

In [3]:
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

### I) Fetching Geo data for Top 5 cities of Oklahoma Stateand creating pandas dataframe

In [4]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="OKC_test")

city_list ={'1':['Oklahoma City',20000],
            '2':['Tulsa',10000],
            '3': ['Norman',5000],
            '4': ['Edmond',6000],
            '5': ['Broken Arrow',10000]}
city=[]
for c in city_list:
    location = geolocator.geocode(city_list[c][0] + ' OK', geometry='geojson')
    city.append([city_list[c][0] ,location.latitude,location.longitude,city_list[c][1]])
    
# define the dataframe columns
column_names = ['City','Latitude', 'Longitude', 'Radius'] 

cities = pd.DataFrame(city, columns = column_names)

OK_data=cities.copy()
OK_data.head(10)

,City,Latitude,Longitude,Radius
0,Oklahoma City,35.472989,-97.517054,20000
1,Tulsa,36.155681,-95.992911,10000
2,Norman,35.222572,-97.439482,5000
3,Edmond,35.657137,-97.464904,6000
4,Broken Arrow,36.052599,-95.790819,10000


In [5]:
#latitude = OK_data[OK_data['City']=='Oklahoma City']['Latitude']
#longitude = OK_data[OK_data['City']=='Oklahoma City']['Longitude']
#print('Oklahoma City Latitude :' , latitude , ' Longitude :', longitude)

# creating Oklahoma City Map using folium

address = 'Oklahoma City, OK'

geolocator = Nominatim(user_agent="okc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Oklahoma City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Oklahoma City are 35.4729886, -97.5170536.


### II) Use Foursquare Expore API to pull Venues data based on cities Latitude and Longitude

Python Function is created to define API call and extract required attributes from API call

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000, limit=100):
    
    # only selecting food, drinks and coffee venues
    section =['food','drinks', 'coffee']
    venues_list=[]
    for name, lat, lng, rds in zip(names, latitudes, longitudes,radius):
        
        #print(name)
        for sName in section:    
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section={}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng,
                sName,
                rds, 
                limit)

            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng,
                v['venue']['id'],
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

            '''
            for venue_list in venues_list:
                for item in venue_list:
                    print (item)
            '''     
            # generating list from venues
            #print([item for venue_list in venues_list for item in venue_list])

    # generating list from venues and creating DataFrame from list
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    #nearby_venues

    nearby_venues.columns = [
                  'City', 
                  'City_Latitude', 
                  'City_Longitude', 
                  'Venue_Id',
                  'Venue_Name', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

#### Making Foursquare Explore API call to fetch Venues

5K meters of radios from city center is used to fetch venue data from foursquare explore API for selection sections e.g. food, drink, and coffee. APIs currently only return 100 records per call.

Explore API with above configuration, has retrieved 916 venues having 86 unique venue categories associated with Food, Drink, and Coffee section

In [7]:
# run to refresh data 

'''
LIMIT = 500  # limit max number of samples API call can provides; currently API limits result to 100
###RADIUS= 5000  # meters radius from center of city

### original df name : OK_venues

OK_restaurant_venues = getNearbyVenues(names=OK_data['City'],
                            latitudes=OK_data['Latitude'],
                            longitudes=OK_data['Longitude'],
                            radius=OK_data['Radius'],
                            #radius=RADIUS,
                            limit=LIMIT
                            )

#print('Oklahoma Venue Shape: ' , OK_restaurant_venues.shape)

# Develop license has limited calls per day. Hence, I have captured call result into csv file and use as many times as needed.
# if no need to make live call, read data from csv

### copy fetched data into file to avoid repetative call. Since unpaid developer account has very limited call available per day
OK_restaurant_venues.to_csv(r'ok_venues.csv',index=None,header=True)
print('Data copied to file')
'''


#columns=['City', 'City_Latitude', 'City_Longitude', 'Venue_Id', 'Venue_Name','Venue_Latitude', 'Venue_Longitude', 'Venue_Category']
OK_restaurant_venues= pd.read_csv(r'ok_venues.csv',header=0)
print('OK_restaurant_venues Shape:',OK_restaurant_venues.shape)
print( 'Venue Count :', OK_restaurant_venues['Venue_Id'].count())
print('There are {} uniques categories.'.format(len(OK_restaurant_venues['Venue_Category'].unique())))
OK_restaurant_venues=OK_restaurant_venues.drop_duplicates()
print( 'Venue Count :', OK_restaurant_venues['Venue_Id'].count())
OK_restaurant_venues.head()

OK_restaurant_venues Shape: (916, 8)
Venue Count : 916
There are 86 uniques categories.
Venue Count : 860


,City,City_Latitude,City_Longitude,Venue_Id,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Oklahoma City,35.472989,-97.517054,4fd22d61e4b0f62cda6526e3,Gogo Sushi,35.478021,-97.521069,Sushi Restaurant
1,Oklahoma City,35.472989,-97.517054,50abcef7e4b09282ea1820cb,Kitchen No. 324,35.470681,-97.516196,Restaurant
2,Oklahoma City,35.472989,-97.517054,5571ccfd498e7431cdec6cc0,Tamashii Ramen House,35.476296,-97.518735,Ramen Restaurant
3,Oklahoma City,35.472989,-97.517054,4e335089a809117a2289831e,Hideaway Pizza,35.476217,-97.514426,Pizza Place
4,Oklahoma City,35.472989,-97.517054,4b37eb0bf964a520974825e3,The Wedge Pizzeria,35.468499,-97.507919,Pizza Place


####  Selecting Restaurant Venues based on Limited Venue Categories
There are few of non-restaurant venues are available in result.I have reviewed all available Venue Categories and decided to captured venues with the following sets of Venue Categories and discarded venue associated with rest of categories 

#### Listing all available unique venue categories from Explore API call.

In [8]:
unwanted_category_list=['Smoke Shop', 
'Winery', 
'Hookah Bar', 
'Comedy Club',
'Arcade',  
'Hotel',
'Bookstore', 
'Bowling Alley', 
'Music Venue',
'Concert Hall', 
'Food Truck', 
'Liquor Store']

# check how many venue do not have category defined
OK_restaurant_venues = OK_restaurant_venues[~OK_restaurant_venues['Venue_Category'].isin(unwanted_category_list)]
print('There are {} uniques categories.'.format(len(OK_restaurant_venues['Venue_Category'].unique())))
print( 'Venue Count :', OK_restaurant_venues['Venue_Id'].count())

There are 74 uniques categories.
Venue Count : 840


#### Use Folium package to display 840 restuarnts on map. 

All the restaurants located in same city are shown on map using same color marker

In [9]:
# setting up color schema for various cities' venue; If number of cities are more than number of listed color, same color get reused
marker_colors = [
    'red',          
    'black',
    'gray',
    'orange',    
    'purple',    
    'lightgray' 
    'pink',
    'green',    
    'blue',    
    'beige',
    'darkred',
    'lightred',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'darkpurple',
    'cadetblue'    
    
]
len(marker_colors)

City_list= list(OK_restaurant_venues['City'].unique())

# create map of New York using latitude and longitude values
map_OK_Restaurant = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, v, vCat, city in zip(OK_restaurant_venues['Venue_Latitude'], OK_restaurant_venues['Venue_Longitude'],OK_restaurant_venues['Venue_Name'],OK_restaurant_venues['Venue_Category'], OK_restaurant_venues['City']):
    label = '{}, {}, {}'.format(vCat, v, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        #color='blue',
        color=marker_colors[int(City_list.index(city)%len(marker_colors))],
        #fill=True,
        #fill_color='#3186cc',
        #fill_color=marker_colors[int(City_list.index(city)%len(marker_colors))],
        fill_opacity=0.6,
        parse_html=False).add_to(map_OK_Restaurant)

map_OK_Restaurant

###  II) Use Foursquare Venue API to pull Venue details e.g. rating, like, tip counts etc.

840 restaurant details have been retrieved by Venue API call. Some of the attributes are not populated for some of the venues. 

Appropriate default values are populated if values for the following attributes are missing in API data set:
    [Attribute] = [default value]
    
    rating = 0
    ratingSignals = 0
    likeCnt = 0
    photosCnt = 0
    tipsCnt = 0
    priceTier=2
    priceTierMsg='Moderate'

In [10]:
# run to refresh data 
# value of n should be increase to load more data
# run the following code twice; first time to fetch details for first 425 rows and second time another 425 rows

'''
LIMIT=50
i=0
n=475  # OK_restaurant_venues.shape[0] # first run 475
#m=917
venues_detail_list=[]

# for first call
for vId in OK_restaurant_venues['Venue_Id'][0:n].items():

# second call
#for vId in OK_restaurant_venues['Venue_Id'][n:m].items():
    if i<=n:
        #print(vId[1])
        #vId='50abcef7e4b09282ea1820cb'
        v_url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&multi={}&limit={}'.format(
                vId[1],
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION,
                'multi',
                LIMIT)

        # make the GET request
        venue_detail_resp = requests.get(v_url)#.json()
        #resp
        venue_detail_data = json.loads(venue_detail_resp.text)
        
        try:
            venue = venue_detail_data['response']['venue']
            
            name = venue["name"]
            if 'rating' not in venue:
                rating = 0
                ratingSignals = 0
            else:
                rating = venue["rating"]        
                ratingSignals = venue["ratingSignals"]
            
            if 'likes' not in venue:
                likeCnt = 0
            else:
                likeCnt = venue["likes"]["count"]
            
            if 'photos' not in venue:
                photosCnt = 0
            else:
                photosCnt = venue["photos"]["count"]
            
            if 'tips' not in venue:
                tipsCnt = 0
            else:
                tipsCnt = venue["tips"]["count"]
                
            if 'price' not in venue:
                priceTier=2 #0
                priceTierMsg='Moderate' #'Unknown'        
            else:
                priceTier = venue["price"]["tier"]
                priceTierMsg =  'Unknown' if pd.isnull(venue["price"]["message"]) else venue["price"]["message"]
            
            #print((name, rating,priceTier,ratingSignals,photosCnt,tipsCnt))
            venues_detail_list.append([(vId[1],name,priceTier,priceTierMsg,rating,ratingSignals,likeCnt,photosCnt,tipsCnt)])
        
        except:
            print(venue_detail_data['meta']['code'])
            print (venue_detail_data['meta']['errorType'])            
       
    
    # increase count to limit number of calls
    i += 1


df_columns=['Venue_Id','Venue_Name','Price_Tier','Price_Tier_Message','Rating','Rating_Signals','Likes_Count','Photos_Count','Tips_Count']
OK_venues_detail_df = pd.DataFrame(venue for venues in venues_detail_list for venue in venues)
OK_venues_detail_df.columns=df_columns

print('OK_venues_detail_df shape:', OK_venues_detail_df.shape)

### copy fetched data into file to avoid repetative call. Since unpaid developer account has very limited call available per day

# for first call 
OK_venues_detail_df.to_csv(r'venues_detail_1.csv',index=None,header=True)

# for second call
#OK_venues_detail_df.to_csv(r'venues_detail_2.csv',index=None,header=True)
print('Data copied to file')
'''

'\nLIMIT=50\ni=0\nn=475  # OK_restaurant_venues.shape[0] # first run 475\n#m=917\nvenues_detail_list=[]\n\n# for first call\nfor vId in OK_restaurant_venues[\'Venue_Id\'][0:n].items():\n\n# second call\n#for vId in OK_restaurant_venues[\'Venue_Id\'][n:m].items():\n    if i<=n:\n        #print(vId[1])\n        #vId=\'50abcef7e4b09282ea1820cb\'\n        v_url = \'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&multi={}&limit={}\'.format(\n                vId[1],\n                CLIENT_ID, \n                CLIENT_SECRET, \n                VERSION,\n                \'multi\',\n                LIMIT)\n\n        # make the GET request\n        venue_detail_resp = requests.get(v_url)#.json()\n        #resp\n        venue_detail_data = json.loads(venue_detail_resp.text)\n        \n        try:\n            venue = venue_detail_data[\'response\'][\'venue\']\n            \n            name = venue["name"]\n            if \'rating\' not in venue:\n                rat

In [11]:
#columns=['Venue_Id', 'Venue_Name', 'Price_Tier', 'Price_Tier_Message', 'Rating','Rating_Signals', 'Likes_Count', 'Photos_Count', 'Tips_Count']
#OK_venues_detail_df= pd.read_csv(r'venues_detail.csv',header=0)

files =['venues_detail_1.csv','venues_detail_2.csv']
OK_venues_detail_dfs = [pd.read_csv(fp,index_col=None) for fp in files]
OK_venues_detail_df = pd.concat(OK_venues_detail_dfs)

# alternative
#OK_venues_detail_df= pd.read_csv(r'venues_detail_1.csv',header=0) # 475

OK_venues_detail_df = OK_venues_detail_df.drop_duplicates()

OK_venues_detail_df = OK_venues_detail_df.groupby(['Venue_Id','Venue_Name']).agg({'Rating':'max', 'Price_Tier':'last', 'Price_Tier_Message':'last', 'Rating':'max', 'Rating_Signals':'max', 'Likes_Count':'max', 'Photos_Count':'max', 'Tips_Count':'max' }).reset_index()

print('OK_venues_detail_df shape:', OK_venues_detail_df.shape)
OK_venues_detail_df.head(10)
OK_venues_detail_df=OK_venues_detail_df.drop('Venue_Name',1)

OK_venues_detail_df shape: (851, 9)


### Gather statistics to generate Report

In [12]:
x = OK_restaurant_venues[['Venue_Id','Venue_Category','City']]
y = x.set_index('Venue_Id').join(OK_venues_detail_df .set_index('Venue_Id'),how='inner').reset_index()
#y.shape()
y['City'].value_counts()

Oklahoma City    279
Tulsa            187
Norman           148
Broken Arrow     134
Edmond            92
Name: City, dtype: int64

In [13]:
y.groupby('Price_Tier_Message')['Venue_Id'].count()

Price_Tier_Message
Cheap             399
Expensive          59
Moderate          368
Very Expensive     14
Name: Venue_Id, dtype: int64

In [14]:
y['Venue_Category'].value_counts().head(15)

Coffee Shop             84
Bar                     65
American Restaurant     52
Donut Shop              52
Mexican Restaurant      49
Café                    46
Burger Joint            42
Pizza Place             37
Brewery                 24
Italian Restaurant      21
Sandwich Place          20
Fast Food Restaurant    17
Bakery                  17
BBQ Joint               17
Asian Restaurant        16
Name: Venue_Category, dtype: int64

In [16]:
#y[y['Venue_Id'].isin( OK_venue_user_like_df [OK_venue_user_like_df ['User_Id']==85997014]['Venue_Id'].tolist())]['Venue_Category'].unique()
#y[y['Venue_Id'].isin( OK_venue_user_like_df [OK_venue_user_like_df ['User_Id']==85997014]['Venue_Id'].tolist())]

### III) Use Foursquare likes API to retrieve all the users who likes 840 Venue

23557 users have like one or more point of interest 916 restaurants.

In [15]:
# run to refresh data 
# value of n should be increase to load more data

'''
LIMIT=200  ### limit number of users who like specific venues

i=0
n=1000
venues_users_likes_list=[]
df_columns=['Venue_Id','User_Id','First_Name','Last_Name','Gender']


for vId in OK_restaurant_venues['Venue_Id'].items():
    if i<=n:
        #print(vId[1])
        #vId='50abcef7e4b09282ea1820cb'
        v_url = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}&multi={}&limit={}'.format(
                vId[1],
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION,
                'multi',
                LIMIT)
        # make the GET request
        user_likes_resp = requests.get(v_url)#.json()
        #resp
        user_likes_data = json.loads(user_likes_resp.text)
        #print( user_likes_data['response']['likes']['count'])
        try:
            if user_likes_data['response']['likes']['count'] >0:
                venues_users_likes_list.append([(vId[1],
                    u['id'],
                    u['firstName'],
                    np.NaN if 'lastName' not in u else u['lastName'],
                    np.NaN if 'gender' not in u else u['gender']) for u in user_likes_data['response']['likes']['items']])

                #print((name, rating,priceTier,ratingSignals,photosCnt,tipsCnt))
                #venues_users_likes_list
                ### working on it 
                #venue_user_like_df = pd.DataFrame(item for likes in venues_users_likes_list for item in likes)
                #venue_user_like_df.append(venues_users_likes_list)        
                #item for venue_list in venues_list for item in venue_list
        except:
            print('Data Issue : ', vId[1])
    
    # increase count to limit number of calls
    i += 1


OK_venue_user_like_df = pd.DataFrame(item for likes in venues_users_likes_list for item in likes)
OK_venue_user_like_df.columns=df_columns

print('OK_venue_user_like_df shape:', OK_venue_user_like_df.shape)


# if no need to make live call, read data from csv
OK_venue_user_like_df.to_csv(r'venue_user_like.csv',index=None,header=True)
print('Data copied to file')
'''

#columns=['Venue_Id', 'User_Id', 'First_Name', 'Last_Name', 'Gender']
OK_venue_user_like_df= pd.read_csv(r'venue_user_like.csv',header=0)
# UserID is now set to 0, so constructing User ID based on FirstName and LastName. It may identify different person as same user if their name are same.
#OK_venue_user_like_df['User_Id']=OK_venue_user_like_df['First_Name'].map(str) + '_' + OK_venue_user_like_df['Last_Name']+'_' + OK_venue_user_like_df['Gender']

OK_venue_user_like_df=OK_venue_user_like_df.drop('Gender',1)
OK_venue_user_like_df=OK_venue_user_like_df.drop_duplicates()

print('OK_venue_user_like_df shape:', OK_venue_user_like_df.shape)
OK_venue_user_like_df.head()


OK_venue_user_like_df shape: (23557, 4)


,Venue_Id,User_Id,First_Name,Last_Name
0,4fd22d61e4b0f62cda6526e3,2415652,Anne Marie,J
1,4fd22d61e4b0f62cda6526e3,508795470,Mathew,M
2,4fd22d61e4b0f62cda6526e3,14201636,Regina,L
3,4fd22d61e4b0f62cda6526e3,35649723,Larry,L
4,4fd22d61e4b0f62cda6526e3,60170550,Sophie,B


Listing users with highest likes counts first. There are few active users, who has provided more than 25 likes for various Venues

In [16]:
user_likes_df = OK_venue_user_like_df.groupby('User_Id')['Venue_Id'].count().reset_index()
user_likes_df.columns =['User_Id','Count']
print('user_likes_df shape:', user_likes_df.shape)
user_likes_df.sort_values(by=('Count'), ascending=False).head(15)

user_likes_df shape: (7390, 2)


,User_Id,Count
3257,27816930,108
7113,420364614,100
6436,142383072,65
404,571110,62
2115,11582716,61
5554,85997014,60
3799,37430283,60
323,430471,57
4577,53631783,56
2290,13210167,55


### Reviewing Active User Likes details

In [17]:
# user with less than 25 likes will be used as active user #85997014  OKC # 13210167 Tulsa
user_likes_df [user_likes_df ['User_Id']==85997014]

,User_Id,Count
5554,85997014,60


In [18]:
OK_venue_user_like_df [OK_venue_user_like_df ['User_Id']==85997014].head()

,Venue_Id,User_Id,First_Name,Last_Name
115,50abcef7e4b09282ea1820cb,85997014,Marty,P
456,594c5032ccad6b590b74fd01,85997014,Marty,P
580,51adf036498e9214e120dfd7,85997014,Marty,P
603,51352262e4b061c206880fd2,85997014,Marty,P
671,55dfae52498e2820dd92aa58,85997014,Marty,P


### Data generation

Since Foursquare likes API doesn't not have user rating provided, I have generating random number between 1-10 and assigning as user rating. 

1 means very low satisfaction

10 mean highly satisfied

In [19]:
OK_venue_user_like_df['User_Rating'] = np.random.randint(1, 11, OK_venue_user_like_df.shape[0])  # 1 to 10
OK_venue_user_like_df.head(10)

,Venue_Id,User_Id,First_Name,Last_Name,User_Rating
0,4fd22d61e4b0f62cda6526e3,2415652,Anne Marie,J,9
1,4fd22d61e4b0f62cda6526e3,508795470,Mathew,M,9
2,4fd22d61e4b0f62cda6526e3,14201636,Regina,L,2
3,4fd22d61e4b0f62cda6526e3,35649723,Larry,L,6
4,4fd22d61e4b0f62cda6526e3,60170550,Sophie,B,1
5,4fd22d61e4b0f62cda6526e3,416925816,PAUL,B,3
6,4fd22d61e4b0f62cda6526e3,327870965,Megan,M,1
7,4fd22d61e4b0f62cda6526e3,1651654,Brianne,R,9
8,4fd22d61e4b0f62cda6526e3,21511133,Maria,C,9
9,4fd22d61e4b0f62cda6526e3,108718780,Alex,W,7


In [20]:
#OK_venue_user_like_df['User_Rating'].min() # 1
#OK_venue_user_like_df['User_Rating'].max() # 10
OK_venue_user_like_df.groupby('User_Rating')['User_Id'].count()

User_Rating
1     2362
2     2348
3     2378
4     2380
5     2294
6     2288
7     2386
8     2312
9     2381
10    2428
Name: User_Id, dtype: int64

#### Limitation Note: 
As you can see, random generator has distributed rating values e.g. 1-10 evenly to all the venues. In real dataset, it may not be the case. Since, data is randomly generated, if you run this set of code again, it may assign different rating value for many users. Result of which, recommendation result will not produce same result. However, if rating data is provided by foursquare API, we will have mostly same venue rating data rated by user, unless user change rating value. Result of which recommendation system will behave constantly

### IV) Restaurant Recommendation System

#### a) Content Based Restaurant Recommendation System using Foursquare Venue Category, Price, Rating and Various Counts

In [21]:
# Venue
OK_restaurant_venues.head()

,City,City_Latitude,City_Longitude,Venue_Id,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Oklahoma City,35.472989,-97.517054,4fd22d61e4b0f62cda6526e3,Gogo Sushi,35.478021,-97.521069,Sushi Restaurant
1,Oklahoma City,35.472989,-97.517054,50abcef7e4b09282ea1820cb,Kitchen No. 324,35.470681,-97.516196,Restaurant
2,Oklahoma City,35.472989,-97.517054,5571ccfd498e7431cdec6cc0,Tamashii Ramen House,35.476296,-97.518735,Ramen Restaurant
3,Oklahoma City,35.472989,-97.517054,4e335089a809117a2289831e,Hideaway Pizza,35.476217,-97.514426,Pizza Place
4,Oklahoma City,35.472989,-97.517054,4b37eb0bf964a520974825e3,The Wedge Pizzeria,35.468499,-97.507919,Pizza Place


In [22]:
# venue Details
OK_venues_detail_df.head()

,Venue_Id,Rating,Price_Tier,Price_Tier_Message,Rating_Signals,Likes_Count,Photos_Count,Tips_Count
0,4a906a8cf964a520b71720e3,9.1,2,Moderate,86,64,48,21
1,4b05867ff964a520216422e3,7.9,2,Moderate,76,47,42,13
2,4b05867ff964a520266422e3,7.6,2,Moderate,40,26,26,12
3,4b05867ff964a520276422e3,7.8,2,Moderate,290,177,358,86
4,4b05867ff964a5202b6422e3,7.8,2,Moderate,82,46,113,30


#### Combining Restaurant and Restaurant Details dataset

Only considering venues which are having venue details provided by Venue API e.g. performing inner join

In [23]:
#OK_Restaurant_Venue_df = OK_restaurant_venues.set_index(['Venue_Id']).join(OK_venues_detail_df.set_index(['Venue_Id']), on='Venue_Id',how='left').reset_index()
OK_Restaurant_Venue_df = OK_restaurant_venues.set_index(['Venue_Id']).join(OK_venues_detail_df.set_index(['Venue_Id']), on='Venue_Id',how='inner').reset_index()

# removing duplicates
OK_Restaurant_Venue_df=OK_Restaurant_Venue_df.drop_duplicates()
print('OK_Restaurant_Venue_df shape : ',OK_Restaurant_Venue_df.shape)
OK_Restaurant_Venue_df.head()

OK_Restaurant_Venue_df shape :  (840, 15)


,Venue_Id,City,City_Latitude,City_Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category,Rating,Price_Tier,Price_Tier_Message,Rating_Signals,Likes_Count,Photos_Count,Tips_Count
0,4fd22d61e4b0f62cda6526e3,Oklahoma City,35.472989,-97.517054,Gogo Sushi,35.478021,-97.521069,Sushi Restaurant,9.3,2,Moderate,98,78,58,19
1,50abcef7e4b09282ea1820cb,Oklahoma City,35.472989,-97.517054,Kitchen No. 324,35.470681,-97.516196,Restaurant,9.0,2,Moderate,272,202,348,73
2,5571ccfd498e7431cdec6cc0,Oklahoma City,35.472989,-97.517054,Tamashii Ramen House,35.476296,-97.518735,Ramen Restaurant,9.0,2,Moderate,58,51,26,6
3,4e335089a809117a2289831e,Oklahoma City,35.472989,-97.517054,Hideaway Pizza,35.476217,-97.514426,Pizza Place,8.9,2,Moderate,142,105,101,30
4,4b37eb0bf964a520974825e3,Oklahoma City,35.472989,-97.517054,The Wedge Pizzeria,35.468499,-97.507919,Pizza Place,9.1,2,Moderate,124,86,103,43


### Perform One Hot Encoding 

Value for categorical attribute will be set to either 1 (if true) or 0 ( if not true). 

Numerical values will be normalized

In [24]:
# check is there any record with null price tier message, otherwise it is not going to work
#OK_Restaurant_Venue_df[pd.isnull(OK_Restaurant_Venue_df['Price_Tier_Message'])]
#OK_Restaurant_Venue_df= OK_Restaurant_Venue_df[~OK_Restaurant_Venue_df['Price_Tier_Message'].isnull()]

# remove restaurant if price tier message is null.
#venuesWithCategory_df = OK_Restaurant_Venue_df[~OK_Restaurant_Venue_df['Price_Tier_Message'].isnull()]

print('OK_Restaurant_Venue_df shape : ',OK_Restaurant_Venue_df.shape)

venuesWithCategory_df = OK_Restaurant_Venue_df.copy()

#For every row in the dataframe, iterate through the list of restaurant and place a 1 into the corresponding column
for index, row in OK_Restaurant_Venue_df.iterrows():
    venuesWithCategory_df.at[index, row['Venue_Category']] = 1
    venuesWithCategory_df.at[index, row['Price_Tier_Message']] = 1

#Filling in the NaN values with 0
venuesWithCategory_df = venuesWithCategory_df.fillna(0)

# normalize various numerical attributes

#venuesWithCategory_df['Rating'] = venuesWithCategory_df['Rating']/(sum(venuesWithCategory_df['Rating']))
#venuesWithCategory_df['Rating_Signals'] = venuesWithCategory_df['Rating_Signals']/(sum(venuesWithCategory_df['Rating_Signals']))
#venuesWithCategory_df['Likes_Count'] = venuesWithCategory_df['Likes_Count']/(sum(venuesWithCategory_df['Likes_Count']))

venuesWithCategory_df['Rating'] = (venuesWithCategory_df['Rating']- venuesWithCategory_df['Rating'].mean()) /(venuesWithCategory_df['Rating'].std())
venuesWithCategory_df['Rating_Signals'] = (venuesWithCategory_df['Rating_Signals']- venuesWithCategory_df['Rating_Signals'].mean()) /(venuesWithCategory_df['Rating_Signals'].std())
venuesWithCategory_df['Likes_Count'] = (venuesWithCategory_df['Likes_Count']- venuesWithCategory_df['Likes_Count'].mean()) /(venuesWithCategory_df['Likes_Count'].std())

'''
venuesWithCategory_df['Moderate'] = venuesWithCategory_df['Moderate']/(sum(venuesWithCategory_df['Moderate']))
venuesWithCategory_df['Expensive'] = venuesWithCategory_df['Expensive']/(sum(venuesWithCategory_df['Moderate']))
venuesWithCategory_df['Very Expensive'] = venuesWithCategory_df['Very Expensive']/(sum(venuesWithCategory_df['Very Expensive']))
venuesWithCategory_df['Cheap'] = venuesWithCategory_df['Cheap']/(sum(venuesWithCategory_df['Cheap']))
'''

venue_category_list = OK_Restaurant_Venue_df['Venue_Category'].unique()
for item in venue_category_list:
    #print (item)
    venuesWithCategory_df[item] = (venuesWithCategory_df[item] - venuesWithCategory_df[item].mean() )/(venuesWithCategory_df[item].std())    
    
price_tier_list = OK_Restaurant_Venue_df['Price_Tier_Message'].unique()
for item in price_tier_list:
    #print (item)
    venuesWithCategory_df[item] = (venuesWithCategory_df[item] - venuesWithCategory_df[item].mean() )/(venuesWithCategory_df[item].std())    

print('venuesWithCategory_df shape : ',venuesWithCategory_df.shape)

venuesWithCategory_df=venuesWithCategory_df.drop('Photos_Count',1).drop('Tips_Count',1).drop('Price_Tier_Message',1).drop('Price_Tier',1)
venuesWithCategory_df.head()

OK_Restaurant_Venue_df shape :  (840, 15)
venuesWithCategory_df shape :  (840, 93)


,Venue_Id,City,City_Latitude,City_Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category,Rating,Rating_Signals,Likes_Count,Sushi Restaurant,Moderate,Restaurant,Ramen Restaurant,Pizza Place,American Restaurant,Burger Joint,Café,Cheap,Expensive,Steakhouse,Very Expensive,Sandwich Place,German Restaurant,Fondue Restaurant,Comfort Food Restaurant,Mexican Restaurant,Taco Place,Food Court,Breakfast Spot,Bagel Shop,New American Restaurant,Vietnamese Restaurant,Asian Restaurant,Latin American Restaurant,BBQ Joint,Brazilian Restaurant,Seafood Restaurant,South American Restaurant,Italian Restaurant,Korean Restaurant,Gastropub,Vegetarian / Vegan Restaurant,Wings Joint,Southern / Soul Food Restaurant,Japanese Restaurant,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Mediterranean Restaurant,French Restaurant,Brewery,Pub,Bar,Cocktail Bar,Lounge,Whisky Bar,Nightclub,Gay Bar,Hotel Bar,Event Space,Cajun / Creole Restaurant,Karaoke Bar,Wine Bar,Nightlife Spot,Tex-Mex Restaurant,Dive Bar,Speakeasy,Ethiopian Restaurant,Sports Bar,Coffee Shop,Tea Room,Bakery,Donut Shop,Moroccan Restaurant,Dessert Shop,Caribbean Restaurant,Deli / Bodega,Middle Eastern Restaurant,Hot Dog Joint,Thai Restaurant,Diner,Fried Chicken Joint,Beer Garden,Tiki Bar,Rock Club,Greek Restaurant,Noodle House
0,4fd22d61e4b0f62cda6526e3,Oklahoma City,35.472989,-97.517054,Gogo Sushi,35.478021,-97.521069,Sushi Restaurant,0.872990,0.985957,1.230795,9.603302,1.131849,-0.109699,-0.048824,-0.214528,-0.256732,-0.229279,-0.240553,-0.950623,-0.274689,-0.13476,-0.130111,-0.156081,-0.059833,-0.034503,-0.048824,-0.248743,-0.084768,-0.059833,-0.115123,-0.084768,-0.104007,-0.120314,-0.139264,-0.048824,-0.143637,-0.034503,-0.120314,-0.048824,-0.160033,-0.034503,-0.115123,-0.048824,-0.06913,-0.034503,-0.059833,-0.143637,-0.059833,-0.098,-0.091615,-0.034503,-0.171396,-0.109699,-0.289432,-0.091615,-0.098,-0.048824,-0.091615,-0.091615,-0.077336,-0.034503,-0.077336,-0.048824,-0.077336,-0.034503,-0.034503,-0.104007,-0.034503,-0.034503,-0.091615,-0.333135,-0.115123,-0.143637,-0.256732,-0.034503,-0.034503,-0.034503,-0.098,-0.034503,-0.059833,-0.091615,-0.098,-0.098,-0.048824,-0.034503,-0.034503,-0.034503,-0.034503
1,50abcef7e4b09282ea1820cb,Oklahoma City,35.472989,-97.517054,Kitchen No. 324,35.470681,-97.516196,Restaurant,0.753194,4.311386,4.568805,-0.104007,1.131849,9.105009,-0.048824,-0.214528,-0.256732,-0.229279,-0.240553,-0.950623,-0.274689,-0.13476,-0.130111,-0.156081,-0.059833,-0.034503,-0.048824,-0.248743,-0.084768,-0.059833,-0.115123,-0.084768,-0.104007,-0.120314,-0.139264,-0.048824,-0.143637,-0.034503,-0.120314,-0.048824,-0.160033,-0.034503,-0.115123,-0.048824,-0.06913,-0.034503,-0.059833,-0.143637,-0.059833,-0.098,-0.091615,-0.034503,-0.171396,-0.109699,-0.289432,-0.091615,-0.098,-0.048824,-0.091615,-0.091615,-0.077336,-0.034503,-0.077336,-0.048824,-0.077336,-0.034503,-0.034503,-0.104007,-0.034503,-0.034503,-0.091615,-0.333135,-0.115123,-0.143637,-0.256732,-0.034503,-0.034503,-0.034503,-0.098,-0.034503,-0.059833,-0.091615,-0.098,-0.098,-0.048824,-0.034503,-0.034503,-0.034503,-0.034503
2,5571ccfd498e7431cdec6cc0,Oklahoma City,35.472989,-97.517054,Tamashii Ramen House,35.476296,-97.518735,Ramen Restaurant,0.753194,0.221491,0.503970,-0.104007,1.131849,-0.109699,20.457302,-0.214528,-0.256732,-0.229279,-0.240553,-0.950623,-0.274689,-0.13476,-0.130111,-0.156081,-0.059833,-0.034503,-0.048824,-0.248743,-0.084768,-0.059833,-0.115123,-0.084768,-0.104007,-0.120314,-0.139264,-0.048824,-0.143637,-0.034503,-0.120314,-0.048824,-0.160033,-0.034503,-0.115123,-0.048824,-0.06913,-0.034503,-0.059833,-0.143637,-0.059833,-0.098,-0.091615,-0.034503,-0.171396,-0.109699,-0.289432,-0.091615,-0.098,-0.048824,-0.091615,-0.091615,-0.077336,-0.034503,-0.077336,-0.048824,-0.077336,-0.034503,-0.034503,-0.104007,-0.034503,-0.034503,-0.091615,-0.333135,-0.115123,-0.143637,-0.256732,-0.034503,-0.034503,-0.034503,-0.098,-0.034503,-0.059833,-0.091615,-0.098,-0.098,-0.048824,-0.034503,-0.034503,-0.034503,-0.0

OK Restaurant dataset has 15 attributes. One hot coding has constructed 60 new attributes e.g. known as future in data science world which will be used by recommendation system.

### Create active user data from user like dataset
User Id is Danny C : User ID 13210167

Danny has logged 54 likes on foursquare for various venues

In [25]:
# create user input data from user like df : Selected user is Danny C # 13210167
userInput=OK_venue_user_like_df[OK_venue_user_like_df['User_Id']==85997014]
#userInput.loc[:,'Like_Rating']= 1

print('userInput before shape : ',userInput.shape)
# removing user reating for any venue which is not in master list ; due to data reterive limits imposed by premium call e.g. limtied venue details
userInput=userInput[userInput.Venue_Id.isin(OK_Restaurant_Venue_df['Venue_Id'].tolist())]
print('userInput after shape : ',userInput.shape)

userInput=userInput.reset_index(drop=True)
#userInput.head(10)

#Filtering out the movies from the input
userVenues = venuesWithCategory_df[venuesWithCategory_df['Venue_Id'].isin(userInput['Venue_Id'].tolist())]
print('userVenues shape : ',userVenues.shape)
#Resetting the index to avoid future issues
userVenues = userVenues.reset_index(drop=True)

#Dropping unnecessary issues due to save memory and to avoid issues
userVenueCategoryTable = userVenues.drop('Venue_Id', 1).drop('City', 1).drop('Venue_Name', 1).drop('Venue_Latitude', 1).drop('Venue_Longitude', 1).drop('Venue_Category',1).drop('City_Latitude',1).drop('City_Longitude',1)
print('userVenueCategoryTable shape : ',userVenueCategoryTable.shape)
#userVenueCategoryTable.head()

userInput before shape :  (60, 5)
userInput after shape :  (60, 5)
userVenues shape :  (60, 89)
userVenueCategoryTable shape :  (60, 81)


In [26]:
# Sushi Restaurant analysis
#OK_Restaurant_Venue_df[OK_Restaurant_Venue_df['Venue_Id']=='4b06ce36f964a5208ff022e3']
#venueCategoryTable.loc['4b06ce36f964a5208ff022e3':'53505cf7498e2a58b5e7aa88']
#userVenues.head()

In [26]:
# index is dropped, but order of record in both dataframes represents same venue.. 
# Dot produt to get weights
# both dataframes must have same rows
userProfile = userVenueCategoryTable.transpose().dot(userInput['User_Rating'])
#The user profile
userProfile.sort_values(ascending=False).head(10)

Likes_Count             368.299011
Rating_Signals          344.775075
Brazilian Restaurant    192.045245
Event Space             163.062492
Rating                  159.234433
American Restaurant     139.188208
Very Expensive          138.698747
German Restaurant       115.237949
Restaurant              103.775164
Steakhouse               93.523223
dtype: float64

In [27]:
#Now let's get the categories of venue in original dataframe
venueCategoryTable = venuesWithCategory_df.copy()
venueCategoryTable.set_index(['Venue_Id'], inplace=True)
print('venueCategoryTable shape : ',venueCategoryTable.shape)
#venueCategoryTable.head()

# drop the unnecessary information
venueCategoryTable.reset_index(drop=True)
venueCategoryTable = venueCategoryTable.drop('City', 1).drop('Venue_Name', 1).drop('Venue_Latitude', 1).drop('Venue_Longitude', 1).drop('Venue_Category', 1)#.drop('Price_Tier_Message', 1).drop('Price_Tier',1)
print('venueCategoryTable shape : ',venueCategoryTable.shape)
#venueCategoryTable.head()

# normalizing data
# number of columns of venueCategoryTable should be same as number of rows in userProfile
recommendationTable = ((venueCategoryTable*userProfile).sum(axis=1))/(userProfile.sum())
#recommendationTable = ((venueCategoryTable*userProfile).sum(axis=1))

print('recommendationTable shape : ',recommendationTable.shape)
recommendationTable.head()

#Sort our recommendations in descending order
recommendationTable = recommendationTable.sort_values(ascending=False)
#Just a peek at the values
#recommendationTable[recommendationTable>0].head(10)

# creating pandas dataframe
recommendationTable_df =pd.DataFrame(recommendationTable, columns=['User_Reco_Rating'])
#recommendationTable_df.reset_index(inplace=True)
print('recommendationTable_df shape : ',recommendationTable_df.shape)
#recommendationTable_df.head()

#The final recommendation table
## All venues including which users liked
## exluding venues which user have alredy liked
#Reco_OK_Venues_df =OK_restaurant_venues.loc[(OK_restaurant_venues['Venue_Id'].isin(recommendationTable_df[recommendationTable_df['User_Reco_Rating']>0].head(10).index.values) )]
OK_Reco_Venues_df =OK_restaurant_venues.loc[(OK_restaurant_venues['Venue_Id'].isin(recommendationTable_df[recommendationTable_df['User_Reco_Rating']>0].head(40).index.values) & ~(OK_restaurant_venues['Venue_Id'].isin(userVenues['Venue_Id'])))]
print('OK_Reco_Venues_df shape : ',OK_Reco_Venues_df.shape)

# joining to copy rating data
OK_Reco_Venues_df = OK_Reco_Venues_df.set_index('Venue_Id').join(recommendationTable_df,how='left').sort_values(by=['User_Reco_Rating'],ascending=False).reset_index()

# ranking for selection
ranks= OK_Reco_Venues_df.groupby(['City','Venue_Category'])['User_Reco_Rating'].rank(ascending=False, method='first')
ranks.name ='User_Reco_Rank'
#ranks
OK_Reco_Venues_df= pd.concat([OK_Reco_Venues_df,ranks], axis=1)
print('OK_Reco_Venues_df Shape :',OK_Reco_Venues_df.shape)
OK_Reco_Venues_df = OK_Reco_Venues_df[OK_Reco_Venues_df['User_Reco_Rank']<=2]
print('OK_Reco_Venues_df Shape (Top 2 ranks) :',OK_Reco_Venues_df.shape)

venueCategoryTable shape :  (840, 88)
venueCategoryTable shape :  (840, 83)
recommendationTable shape :  (840,)
recommendationTable_df shape :  (840, 1)
OK_Reco_Venues_df shape :  (21, 8)
OK_Reco_Venues_df Shape : (21, 10)
OK_Reco_Venues_df Shape (Top 2 ranks) : (21, 10)


In [28]:
OK_Reco_Venues_df.sort_values(by=['Venue_Category','User_Reco_Rank','User_Reco_Rating'], ascending=[True,True,False])

,Venue_Id,City,City_Latitude,City_Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category,User_Reco_Rating,User_Reco_Rank
6,4b1ae56bf964a52029f423e3,Oklahoma City,35.472989,-97.517054,BJ's Restaurant & Brewhouse,35.610272,-97.557901,American Restaurant,2.835065,1.0
10,4da23905b521224b059905ee,Norman,35.222572,-97.439482,The Garage,35.222496,-97.440340,American Restaurant,2.123188,1.0
19,4ca3ce207f84224b0614c858,Tulsa,36.155681,-95.992911,Polo Grill,36.131987,-95.962965,American Restaurant,1.722776,1.0
20,4c7e87c701df37040084e3ac,Edmond,35.657137,-97.464904,Charleston's Restaurant,35.622627,-97.487834,American Restaurant,1.709526,1.0
15,4b5a5336f964a52051bd28e3,Norman,35.222572,-97.439482,BJ's Restaurant & Brewhouse,35.216403,-97.487469,American Restaurant,1.806183,2.0
17,5103edfbe4b0700e0c313fba,Oklahoma City,35.472989,-97.517054,BJ's Restaurant & Brewhouse,35.461583,-97.652230,American Restaurant,1.752346,2.0
16,4b0b3b15f964a5201a2f23e3,Oklahoma City,35.472989,-97.517054,Iron Star Urban BBQ,35.507954,-97.525087,BBQ Joint,1.770429,1.0
7,4cae666bb4b0a35d4ffb38ce,Tulsa,36.155681,-95.992911,Fassler Hall,36.154783,-95.986418,Beer Garden,2.824209,1.0
12,4f0c8478e4b0c67b44fc79c8,Oklahoma City,35.472989,-97.517054,S&B's Burger Joint,35.476904,-97.513071,Burger Joint,1.952676,1.0
9,4b6c50a0f964a520fe2f2ce3,Tulsa,36.155681,-95.992911,Tally's Good Food Café,36.147658,-95.922659,Diner,2.183428,1.0


In [29]:
# create map of New York using latitude and longitude values
ok_user_reco_venue_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map

for lat, lng, v, vc in zip(OK_restaurant_venues['Venue_Latitude'], OK_restaurant_venues['Venue_Longitude'], OK_restaurant_venues['Venue_Name'],OK_restaurant_venues['Venue_Category']):
    label = '{} ({})'.format(v, vc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(ok_user_reco_venue_map)  

for lat, lng, v, vc in zip(OK_Reco_Venues_df['Venue_Latitude'], OK_Reco_Venues_df['Venue_Longitude'], OK_Reco_Venues_df['Venue_Name'],OK_Reco_Venues_df['Venue_Category']):
    label = '{} ({})'.format(v, vc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(ok_user_reco_venue_map)  

for lat, lng, v, vc in zip(userVenues['Venue_Latitude'], userVenues['Venue_Longitude'], userVenues['Venue_Name'],userVenues['Venue_Category']):
    label = '{} ({})'.format(v, vc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(ok_user_reco_venue_map)
    
ok_user_reco_venue_map

### VII) Collaborative filtering based Venue Recommendation system using Foursquare Venue, Menu, and Like dataset 

In [30]:
print('userInput Shape :',userInput.shape)
#userInput.head()
userSubset = OK_venue_user_like_df[OK_venue_user_like_df['Venue_Id'].isin(userInput['Venue_Id'].tolist())]
print('userSubset Shape :',userSubset.shape)
userSubset.head(10)

userInput Shape : (60, 5)
userSubset Shape : (3968, 5)


,Venue_Id,User_Id,First_Name,Last_Name,User_Rating
67,50abcef7e4b09282ea1820cb,458031240,Ian,M,6
68,50abcef7e4b09282ea1820cb,496796563,Addison,B,8
69,50abcef7e4b09282ea1820cb,31221964,Jeff,S,10
70,50abcef7e4b09282ea1820cb,500008002,Mike,M,9
71,50abcef7e4b09282ea1820cb,20194101,April,T,9
72,50abcef7e4b09282ea1820cb,452519568,Laurie,M,3
73,50abcef7e4b09282ea1820cb,98506745,Krista,M,5
74,50abcef7e4b09282ea1820cb,29585479,TheGrtNecessity,NaN,9
75,50abcef7e4b09282ea1820cb,132002700,Katie,C,2
76,50abcef7e4b09282ea1820cb,420364614,Matthew,S,5


In [31]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['User_Id'])
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x), reverse=True)
#userSubset.groupby(['User_Id']).transform(len).sort_values('Venue_Id', ascending=False)
userSubsetGroup[0:5]
#type(userSubsetGroup) # list

# consider only first 100 subgroups
userSubsetGroup = userSubsetGroup[0:100]
len(userSubsetGroup)

100

In [32]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #if name == 17:
        #Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='Venue_Id')
        userInput = userInput.sort_values(by='Venue_Id')
        #Get the N for the formula
        nRatings = len(group)
        #print("N :", nRatings)
        #Get the review scores for the Venue that they both have in common
        temp_df = userInput[userInput['Venue_Id'].isin(group['Venue_Id'].tolist())]
        #print(temp_df)
        #And then store them in a temporary buffer variable in a list format to facilitate future calculations
        tempRatingList = temp_df['User_Rating'].tolist()
        #print(tempRatingList)
        #Let's also put the current user group reviews in a list format
        tempGroupList = group['User_Rating'].tolist()
        #print(tempGroupList)
        #Now let's calculate the pearson correlation between two users, so called, x and y
        Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
        Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
        Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

        #If the denominator is different than zero, then divide, else, 0 correlation.
        if Sxx != 0 and Syy != 0:
            pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearsonCorrelationDict[name] = 0

In [33]:
pearsonCorrelationDict.items()

dict_items([(381, -0.18898223650461363), (1808, 0), (2095, 0), (2764, -1.0), (5043, 0), (8313, -1.0), (8407, 0), (11343, 0), (11631, 0), (18541, 0), (21143, 0), (23608, 0), (25116, -0.08104408984731042), (29038, -0.09078412990032038), (31673, 0.7857142857142858), (36551, 1.0), (36981, 0), (37881, -0.18898223650461535), (39509, 0), (46247, -0.026659558398736394), (46936, 0), (53475, -0.904194430179463), (57655, 0), (60564, 0), (61943, 0), (63960, -1.0), (66667, 0), (69093, 1.0), (85182, 0), (95156, 0), (100065, 0), (103709, -1.0), (104355, 0), (106363, 0), (107931, 1.0), (109965, -0.7292845505553168), (110577, 0), (110707, 0.14285714285714393), (111187, 0), (112318, -0.4749448516257124), (112854, 0), (117033, 0), (117655, 0), (118899, -0.2773500981126149), (118977, 0), (119233, -0.16445305702801818), (120866, -1.0), (122892, 0.9365858115816941), (122911, 0), (126052, 1.0), (130508, 0), (130669, 0.6949796831257331), (133191, -1.0), (138553, -0.7953559537197221), (139923, 0), (142526, 1.0

In [34]:
# create pandas dataframe to store pearson correlation data
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['User_Id'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,User_Id
0,-0.188982,381
1,0.000000,1808
2,0.000000,2095
3,-1.000000,2764
4,0.000000,5043


In [35]:
#Now let's get the top 50 users that are most similar to the input.
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
#topUsers[topUsers['similarityIndex']!=0]['User_Id'].value_counts()
topUsers.head()

,similarityIndex,User_Id
34,1.0,107931
49,1.0,126052
83,1.0,247427
27,1.0,69093
55,1.0,142526


In [36]:
# use all the venues data include venues active user like. Later venus which active users like will be separated from the venues which active user not yet visisted and used for recommendation
topUsersRating=topUsers.merge(OK_venue_user_like_df, left_on='User_Id', right_on='User_Id', how='inner')
print('topUsersRating Shape :',topUsersRating.shape)
#topUsersRating[topUsersRating['Venue_Id']=='56e9976c498eca239fd44b82']
topUsersRating.head(10)

topUsersRating Shape : (311, 6)


,similarityIndex,User_Id,Venue_Id,First_Name,Last_Name,User_Rating
0,1.0,107931,56e9976c498eca239fd44b82,Beau,W,3
1,1.0,107931,4f6c98357beb467b5c0306ba,Beau,W,7
2,1.0,107931,4bb369b84019a593d4de37b8,Beau,W,8
3,1.0,126052,4f5547f7e4b0d0432f65e8dc,Craig,B,5
4,1.0,126052,4dc41571814d503d3b15ee19,Craig,B,10
5,1.0,126052,4b66053df964a520d90e2be3,Craig,B,9
6,1.0,126052,4b758d4ff964a520a9132ee3,Craig,B,2
7,1.0,126052,55c54b58498e3f3d60788671,Craig,B,2
8,1.0,126052,4bd8dcda11dcc9282e30f833,Craig,B,1
9,1.0,247427,4f0c8478e4b0c67b44fc79c8,Steve,R,3


In [37]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['User_Rating']
topUsersRating.head(12)

,similarityIndex,User_Id,Venue_Id,First_Name,Last_Name,User_Rating,weightedRating
0,1.0,107931,56e9976c498eca239fd44b82,Beau,W,3,3.0
1,1.0,107931,4f6c98357beb467b5c0306ba,Beau,W,7,7.0
2,1.0,107931,4bb369b84019a593d4de37b8,Beau,W,8,8.0
3,1.0,126052,4f5547f7e4b0d0432f65e8dc,Craig,B,5,5.0
4,1.0,126052,4dc41571814d503d3b15ee19,Craig,B,10,10.0
5,1.0,126052,4b66053df964a520d90e2be3,Craig,B,9,9.0
6,1.0,126052,4b758d4ff964a520a9132ee3,Craig,B,2,2.0
7,1.0,126052,55c54b58498e3f3d60788671,Craig,B,2,2.0
8,1.0,126052,4bd8dcda11dcc9282e30f833,Craig,B,1,1.0
9,1.0,247427,4f0c8478e4b0c67b44fc79c8,Steve,R,3,3.0


In [38]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('Venue_Id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print('tempTopUsersRating Shape :',tempTopUsersRating.shape)
tempTopUsersRating.head()

tempTopUsersRating Shape : (188, 2)


,sum_similarityIndex,sum_weightedRating
Venue_Id,,
4b05867ff964a520216422e3,0.572675,5.726753
4b05867ff964a520276422e3,1.000000,3.000000
4b05867ff964a5202b6422e3,0.785714,1.571429
4b05867ff964a520346422e3,0.000000,0.000000
4b058681f964a520776422e3,0.785714,3.142857


In [39]:
#Creates an empty dataframe
CFR_recommendation_df = pd.DataFrame()
#Now we take the weighted average
CFR_recommendation_df['WA_Reco_Score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
CFR_recommendation_df['Venue_Id'] = tempTopUsersRating.index
#recommendation_df.head()

CFR_recommendation_df = CFR_recommendation_df.sort_values(by='WA_Reco_Score', ascending=False)
#recommendation_df=recommendation_df[~recommendation_df['weighted average recommendation score'].isnull()]
CFR_recommendation_df=CFR_recommendation_df.reset_index(drop=True).set_index('Venue_Id')
print('CFR_recommendation_df Shape :',CFR_recommendation_df.shape)
CFR_recommendation_df.head()

CFR_recommendation_df Shape : (188, 1)


,WA_Reco_Score
Venue_Id,
4b05867ff964a520216422e3,10.0
5424d068498e9bc1c2913d28,10.0
4f621197e4b0c2e657bbb453,10.0
5055235fe4b0ff6b62181e51,10.0
4b41070af964a520a9bf25e3,10.0


In [40]:
#The final recommendation table
## All vanues including which users liked
#Reco_OKC_Venues_df =OKC_food_venues.loc[OKC_food_venues['Venue_Id'].isin(recommendationTable_df[recommendationTable_df>0].head(10).keys())]
## exluding venues which user have alredy liked
OK_CFR_Reco_Venues_df =OK_restaurant_venues.loc[(OK_restaurant_venues['Venue_Id'].isin(CFR_recommendation_df[CFR_recommendation_df['WA_Reco_Score']>0].head(40).index.values) & ~(OK_restaurant_venues['Venue_Id'].isin(userVenues['Venue_Id'])))]
#Reco_OKC_Venues_df =OK_restaurant_venues.loc[(OK_restaurant_venues['Venue_Id'].isin(CFR_recommendation_df[CFR_recommendation_df['WA_Reco_Score']>0].head(10).index.values) )]

# joining to copy rating data
OK_CFR_Reco_Venues_df = OK_CFR_Reco_Venues_df.set_index('Venue_Id').join(CFR_recommendation_df,on='Venue_Id', how='left').sort_values(by=['WA_Reco_Score'],ascending=False).reset_index()

# ranking for selection

ranks= OK_CFR_Reco_Venues_df.groupby(['City','Venue_Category'])['WA_Reco_Score'].rank(ascending=False, method='first')
ranks.name ='WA_Reco_Rank'
#ranks

OK_CFR_Reco_Venues_df= pd.concat([OK_CFR_Reco_Venues_df,ranks], axis=1)
print('OK_CFR_Reco_Venues_df Shape :',OK_CFR_Reco_Venues_df.shape)
OK_CFR_Reco_Venues_df = OK_CFR_Reco_Venues_df[OK_CFR_Reco_Venues_df['WA_Reco_Rank']<=2]
print('OK_CFR_Reco_Venues_df Shape :',OK_CFR_Reco_Venues_df.shape)
#OK_CFR_Reco_Venues_df

OK_CFR_Reco_Venues_df Shape : (29, 10)
OK_CFR_Reco_Venues_df Shape : (29, 10)


In [41]:
OK_CFR_Reco_Venues_df.sort_values(by=['Venue_Category','WA_Reco_Score','WA_Reco_Rank'], ascending=[True,False,True])

,Venue_Id,City,City_Latitude,City_Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category,WA_Reco_Score,WA_Reco_Rank
22,4da23905b521224b059905ee,Norman,35.222572,-97.439482,The Garage,35.222496,-97.440340,American Restaurant,8.000000,1.0
18,4babf789f964a52012da3ae3,Oklahoma City,35.472989,-97.517054,Bedlam BBQ,35.522051,-97.502132,BBQ Joint,8.332226,1.0
2,4b510161f964a520283d27e3,Tulsa,36.155681,-95.992911,Panera Bread,36.140296,-95.968668,Bakery,10.000000,1.0
19,55ef5cb3498e1af21fcfd735,Edmond,35.657137,-97.464904,Eggington's,35.668072,-97.494787,Breakfast Spot,8.000000,1.0
7,5055235fe4b0ff6b62181e51,Oklahoma City,35.472989,-97.517054,Tucker's Onion Burgers,35.530953,-97.534464,Burger Joint,10.000000,1.0
28,4b0d865bf964a520334a23e3,Edmond,35.657137,-97.464904,Flatire Burgers,35.658635,-97.475855,Burger Joint,7.675244,1.0
23,5391ea47498e574c69ac18d1,Oklahoma City,35.472989,-97.517054,Aspen Coffee,35.637880,-97.486760,Coffee Shop,8.000000,1.0
27,5391ea47498e574c69ac18d1,Edmond,35.657137,-97.464904,Aspen Coffee,35.637880,-97.486760,Coffee Shop,8.000000,1.0
4,5424d068498e9bc1c2913d28,Oklahoma City,35.472989,-97.517054,The R&J Supper Club,35.477940,-97.518715,Comfort Food Restaurant,10.000000,1.0
14,4b79d577f964a5200d152fe3,Tulsa,36.155681,-95.992911,Dilly Diner,36.155826,-95.986708,Deli / Bodega,9.000000,1.0


In [42]:
#OK_CFR_Reco_Venues_df.sort_values(by='WA_Reco_Rank', ascending=False)
OK_CFR_Reco_Venues_df[OK_CFR_Reco_Venues_df['City']=='Oklahoma City'].sort_values(by=['WA_Reco_Score','WA_Reco_Rank'], ascending=False)

,Venue_Id,City,City_Latitude,City_Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category,WA_Reco_Score,WA_Reco_Rank
0,4b05867ff964a520216422e3,Oklahoma City,35.472989,-97.517054,Flip's Wine Bar & Trattoria,35.532009,-97.530022,Wine Bar,10.000000,1.0
4,5424d068498e9bc1c2913d28,Oklahoma City,35.472989,-97.517054,The R&J Supper Club,35.477940,-97.518715,Comfort Food Restaurant,10.000000,1.0
5,4b41070af964a520a9bf25e3,Oklahoma City,35.472989,-97.517054,Zorba's Mediterranean Cuisine,35.532412,-97.565230,Mediterranean Restaurant,10.000000,1.0
6,4b09e504f964a520531f23e3,Oklahoma City,35.472989,-97.517054,Tokyo Japanese Restaurant,35.548308,-97.529606,Japanese Restaurant,10.000000,1.0
7,5055235fe4b0ff6b62181e51,Oklahoma City,35.472989,-97.517054,Tucker's Onion Burgers,35.530953,-97.534464,Burger Joint,10.000000,1.0
8,4b12e5f8f964a5206b9023e3,Oklahoma City,35.472989,-97.517054,The Red Cup,35.501570,-97.533640,Vegetarian / Vegan Restaurant,10.000000,1.0
9,576471e3498e440ef9f53723,Oklahoma City,35.472989,-97.517054,Mary Eddy's,35.467694,-97.527462,Restaurant,10.000000,1.0
10,4bce47b2fb84c9b6139e233e,Oklahoma City,35.472989,-97.517054,Stella's Modern Italian Cuisine,35.480262,-97.521494,Italian Restaurant,9.500000,1.0
11,4cf92ea6d76da35d1562cb2c,Oklahoma City,35.472989,-97.517054,Chick-fil-A,35.535389,-97.566106,Fast Food Restaurant,9.000000,1.0
12,4b0b0311f964a5208c2b23e3,Oklahoma City,35.472989,-97.517054,Golden Phoenix,35.497220,-97.534145,Vietnamese Restaurant,9.000000,1.0


In [43]:
OK_Reco_Venues_df[OK_Reco_Venues_df['City']=='Oklahoma City'].sort_values(by=['User_Reco_Rating','User_Reco_Rank'], ascending=False)

,Venue_Id,City,City_Latitude,City_Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category,User_Reco_Rating,User_Reco_Rank
2,4b8ab246f964a520787a32e3,Oklahoma City,35.472989,-97.517054,Ingrid's,35.507934,-97.552329,German Restaurant,4.131156,1.0
3,5179b92de4b069b816cea54c,Oklahoma City,35.472989,-97.517054,Whiskey Cake Kitchen & Bar,35.523028,-97.543056,New American Restaurant,4.044934,1.0
4,4b0da7edf964a520454d23e3,Oklahoma City,35.472989,-97.517054,Big Truck Tacos,35.493087,-97.522329,Taco Place,3.160927,1.0
6,4b1ae56bf964a52029f423e3,Oklahoma City,35.472989,-97.517054,BJ's Restaurant & Brewhouse,35.610272,-97.557901,American Restaurant,2.835065,1.0
8,4b2fe3d5f964a5200cf224e3,Oklahoma City,35.472989,-97.517054,Red PrimeSteak,35.472071,-97.514061,Steakhouse,2.642391,1.0
12,4f0c8478e4b0c67b44fc79c8,Oklahoma City,35.472989,-97.517054,S&B's Burger Joint,35.476904,-97.513071,Burger Joint,1.952676,1.0
16,4b0b3b15f964a5201a2f23e3,Oklahoma City,35.472989,-97.517054,Iron Star Urban BBQ,35.507954,-97.525087,BBQ Joint,1.770429,1.0
17,5103edfbe4b0700e0c313fba,Oklahoma City,35.472989,-97.517054,BJ's Restaurant & Brewhouse,35.461583,-97.652230,American Restaurant,1.752346,2.0


In [44]:
both_reco_df = pd.merge(OK_CFR_Reco_Venues_df, OK_Reco_Venues_df, how='inner', on=['Venue_Id'])
both_reco_df.head(10)

,Venue_Id,City_x,City_Latitude_x,City_Longitude_x,Venue_Name_x,Venue_Latitude_x,Venue_Longitude_x,Venue_Category_x,WA_Reco_Score,WA_Reco_Rank,City_y,City_Latitude_y,City_Longitude_y,Venue_Name_y,Venue_Latitude_y,Venue_Longitude_y,Venue_Category_y,User_Reco_Rating,User_Reco_Rank
0,4da23905b521224b059905ee,Norman,35.222572,-97.439482,The Garage,35.222496,-97.440340,American Restaurant,8.0,1.0,Norman,35.222572,-97.439482,The Garage,35.222496,-97.440340,American Restaurant,2.123188,1.0
1,4b8ab246f964a520787a32e3,Oklahoma City,35.472989,-97.517054,Ingrid's,35.507934,-97.552329,German Restaurant,8.0,1.0,Oklahoma City,35.472989,-97.517054,Ingrid's,35.507934,-97.552329,German Restaurant,4.131156,1.0


In [45]:
# create map of New York using latitude and longitude values
ok_cfr_user_reco_venue_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map

for lat, lng, v, vc in zip(OK_restaurant_venues['Venue_Latitude'], OK_restaurant_venues['Venue_Longitude'], OK_restaurant_venues['Venue_Name'],OK_restaurant_venues['Venue_Category']):
    label = '{} ({})'.format(v, vc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        #fill=True,
        #fill_color='lightred',
        fill_opacity=0.7,
        parse_html=False).add_to(ok_cfr_user_reco_venue_map)  

for lat, lng, v, vc in zip(OK_Reco_Venues_df['Venue_Latitude'], OK_Reco_Venues_df['Venue_Longitude'], OK_Reco_Venues_df['Venue_Name'],OK_Reco_Venues_df['Venue_Category']):
    label = '{} ({})'.format(v, vc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='brown',
        #fill=True,
        #fill_color='lightblue',
        fill_opacity=0.7,
        parse_html=False).add_to(ok_cfr_user_reco_venue_map)  
    
for lat, lng, v, vc in zip(OK_CFR_Reco_Venues_df['Venue_Latitude'], OK_CFR_Reco_Venues_df['Venue_Longitude'], OK_CFR_Reco_Venues_df['Venue_Name'],OK_CFR_Reco_Venues_df['Venue_Category']):
    label = '{} ({})'.format(v, vc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        #fill=True,
        #fill_color='lightbrown',
        fill_opacity=0.7,
        parse_html=False).add_to(ok_cfr_user_reco_venue_map)  

for lat, lng, v, vc in zip(both_reco_df['Venue_Latitude_x'], both_reco_df['Venue_Longitude_x'], both_reco_df['Venue_Name_x'],both_reco_df['Venue_Category_x']):
    label = '{} ({})'.format(v, vc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='brown',
        fill_opacity=0.7,
        parse_html=False).add_to(ok_cfr_user_reco_venue_map)  

for lat, lng, v, vc in zip(userVenues['Venue_Latitude'], userVenues['Venue_Longitude'], userVenues['Venue_Name'],userVenues['Venue_Category']):
    label = '{} ({})'.format(v, vc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='lightgreen',
        fill_opacity=0.7,
        parse_html=False).add_to(ok_cfr_user_reco_venue_map)  

ok_cfr_user_reco_venue_map